In [1]:
!pip install -q ivy
!git clone https://github.com/unifyai/models.git --depth 1

# Installing models package from cloned repository! 😄
!cd models/ && pip install .

!python3 -m pip install torchvision

exit()

Cloning into 'models'...


Processing c:\users\administrator\desktop\project\hackthons\ivy-examples\models
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ivy-models: filename=ivy_models-1.1.9-py3-none-any.whl size=76111 sha256=49c5e8766d346308d2c5196102b890040eea5ef00919bac66fa50d80a90ea2d6
  Stored in directory: C:\Users\Administrator\AppData\Local\Temp\2\pip-ephem-wheel-cache-woeqvplp\wheels\be\36\3d\eb4e543ce26bda98974127bdb6a4f1d0d8345328beb387f1bb
Successfully built ivy-models


'python3' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import ivy
import torch

In [2]:
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

'wget' is not recognized as an internal or external command,
operable program or batch file.


FileNotFoundError: [Errno 2] No such file or directory: 'imagenet_classes.txt'

In [ ]:
# Load an image from the examples folder
!wget https://raw.githubusercontent.com/unifyai/models/master/images/cat.jpg
filename = "cat.jpg"

In [ ]:
# Preprocess torch image
from torchvision import transforms
from PIL import Image

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )])
torch_img = Image.open(filename)
torch_img = preprocess(torch_img)
torch_img = torch.unsqueeze(torch_img, 0)

In [ ]:
from IPython.display import Image, display
display(Image(filename))

In [ ]:
from torchvision.models import resnet34, ResNet34_Weights

torch_resnet_34 = resnet34(weights=ResNet34_Weights.IMAGENET1K_V1).to("cuda")
torch_resnet_34.eval()

In [ ]:
# initialize ivy model with ivy resnet weights
from ivy_models.resnet import resnet_34

ivy_resnet_34 = resnet_34(pretrained=True)

In [ ]:
ivy.set_backend('torch')

img = ivy.asarray(torch_img.permute((0, 2, 3, 1)), dtype="float32", device="gpu:0")
ivy_resnet_34.compile(args=(img,))

In [ ]:
# Run inference on torch model
torch_output = torch.softmax(torch_resnet_34(torch_img.cuda()), dim=1)
torch_classes = torch.argsort(torch_output[0], descending=True)[:3]
torch_logits = torch.take(torch_output[0], torch_classes)

print("Indices of the top 3 classes are:", torch_classes)
print("Logits of the top 3 classes are:", torch_logits)
print("Categories of the top 3 classes are:", [categories[i] for i in torch_classes])

In [ ]:
# Run inference on ivy model
output = ivy.softmax(ivy_resnet_34(img))  # pass the image to the model
classes = ivy.argsort(output[0], descending=True)[:3]  # get the top 3 classes
logits = ivy.gather(output[0], classes)  # get the logits

print("Indices of the top 3 classes are:", classes)
print("Logits of the top 3 classes are:", logits)
print("Categories of the top 3 classes are:", [categories[i] for i in classes.to_list()])

In [ ]:
# Compare the results and model inference times
from torchvision.models import resnet50, ResNet50_Weights

torch_resnet_50 = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2).to("cuda")
torch_resnet_50.eval()

In [ ]:
from ivy_models.resnet import resnet_50

ivy_resnet_50 = resnet_50(pretrained=True)

In [ ]:
ivy_resnet_50.compile(args=(img,))

In [ ]:
torch_output = torch.softmax(torch_resnet_50(torch_img.cuda()), dim=1)
torch_classes = torch.argsort(torch_output[0], descending=True)[:3]
torch_logits = torch.take(torch_output[0], torch_classes)

print("Indices of the top 3 classes are:", torch_classes)
print("Logits of the top 3 classes are:", torch_logits)
print("Categories of the top 3 classes are:", [categories[i] for i in torch_classes])

In [ ]:
output = ivy.softmax(ivy_resnet_50(ivy.asarray(img)))  # pass the image to the model
classes = ivy.argsort(output[0], descending=True)[:3]  # get the top 3 classes
logits = ivy.gather(output[0], classes)  # get the logits

print("Indices of the top 3 classes are:", classes)
print("Logits of the top 3 classes are:", logits)
print("Categories of the top 3 classes are:", [categories[i] for i in classes.to_list()])